<a href="https://colab.research.google.com/github/GunduSriBhanu/LLM_Vector_Stores/blob/main/LLama_Index_Neo4j_Vector_Store_Property_Graphs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --quiet llama-index llama-index-graph-stores-neo4j llama-index-program-openai llama-index-llms-openai langchain-groq

In [ ]:
import os

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from google.colab import userdata

In [ ]:
groq_api_key=userdata.get('groq_api_key')
os.environ['OPENAI_API_KEY'] =userdata.get('OPENAI_API_KEY')

In [ ]:
#Graph DB Configuration

NEO4J_URI = userdata.get('NEO4J_URI')
NEO4J_USERNAME = userdata.get('NEO4J_USERNAME')
NEO4J_PASSWORD = userdata.get('NEO4J_PASSWORD')


In [ ]:
from llama_index.graph_stores.neo4j import Neo4jPGStore


graph_store = Neo4jPGStore(
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    url=NEO4J_URI,
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from llama_index.core import Document

news = pd.read_csv("/content/construction.csv")
documents = [Document(text=f"{row['Manufacturer']} provides part number {row['Part_Number']} for {row['Purchase_Description']}",metadata={"copper":["cu","Type 1"]}) for i, row in news.iterrows()]
news.head()

,PoNumber,Purchase_Description,Manufacturer,Part_Number,Vendor_Description
0,19259.0005,2 by 2 by 3/4 inch copper tee,OTHER MANUFACTURERS,CTKKF,2X2X3/4 WROT CXCXC TEE
1,19259.0005,2 inch lf union,IMPORT MANUFACTURERS,IBRLF125UK,LF 2 BRS 125# UNION
2,19259.0005,3/4 inch copper adators,OTHER MANUFACTURERS,CMAF,3/4 WROT CXM ADPT
3,19259.0005,3/4 inch ball valves,NIBCO,NT58580LFHCF,LF 3/4 BRZ BV 2PC FP
4,19259.0005,2 inch ball valves,MILWAUKEE VALVE,MUPBA400K,LF 2 BRZ CP 600# THRD FP BV


In [ ]:
documents[0]

In [ ]:
documents = []
ids = []

for i, row in news.iterrows():
    doc = Document(
        text=f"{row['Manufacturer']} provides part number {row['Part_Number']} for {row['Purchase_Description']}",
        metadata={"copper": ["cu", "Type 1"]}
    )
    documents.append(doc)
    ids.append(doc.id_)  # Extract the id_ from the Document

# Add the documents to the DataFrame in a new column
news['Document'] = documents
news.head()

In [ ]:
news['triplet_source_id'] = ids

In [ ]:
news['Document'][0]

In [ ]:
# Load the Excel file
file_path = '/content/DatabaseExport242424.xlsx'
df = pd.read_excel(file_path)
df

In [ ]:
from google.colab import userdata


In [ ]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from langchain_groq import ChatGroq
from llama_index.core.indices.property_graph import DynamicLLMPathExtractor
from llama_index.core import PropertyGraphIndex
groq_api_key=userdata.get('groq_api_key')
os.environ['OPENAI_API_KEY'] =userdata.get('OPENAI_API_KEY')
llm = OpenAI(model="gpt-3.5-turbo-0125", temperature=0.0)
#llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
embed_model = OpenAIEmbedding(model_name="text-embedding-3-small")

In [ ]:
docs = [
    Document(
        text=f"{row['Component_Long_Description']} with size of {row['Size']}, weight {row['Weight']} and thickness {row['Thickness']}",
        metadata={
            "Material_Code": {
               "value": "American Society for Testing and Materials",
              "abbreviation": "ASTM",

             }

          }
    )
    for i, row in df.iterrows()
]
docs

[Document(id_='dcdf41e1-0bf8-40eb-8a68-8e54b9e650fe', embedding=None, metadata={'Material_Code': {'value': 'American Society for Testing and Materials', 'abbreviation': 'ASTM'}}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="Tube Hard Type L CU ASTM B88 C12200 with size of 1/4'', weight 0.0 and thickness 0.0", mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'),
 Document(id_='8062a9d0-9158-4cc3-aedb-3e1be271f40b', embedding=None, metadata={'Material_Code': {'value': 'American Society for Testing and Materials', 'abbreviation': 'ASTM'}}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="Tube Hard Type L CU ASTM B88 C12200 with size of 3/8'', weight 0.0 and thickness 0.0", mimetype='text/plain', start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_templ

In [ ]:
df.shape

(55276, 46)

In [ ]:
kg_extractor = DynamicLLMPathExtractor(
    llm=llm,
    max_triplets_per_chunk=20,
    num_workers=6,
    allowed_entity_types=["MATERIAL", "MATERIAL_TYPE","PART_NUMBER","MANUFACTURER","DIMENSION","PRODUCT"],
    allowed_relation_types=["HAS", "HAS", "PART_OF","MENTIONS","RESPONDS","SIZE_OF","SIZE_IN_INCHES","PRODUCES"],
)

index = PropertyGraphIndex.from_documents(
    documents[:50],
    kg_extractors=[kg_extractor],
    llm=llm,
    embed_model=embed_model,
    property_graph_store=graph_store,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/39 [00:00<?, ?it/s]

Extracting and inferring knowledge graph from text: 100%|██████████| 39/39 [00:20<00:00,  1.93it/s]
Generating embeddings: 100%|██████████| 3/3 [00:01<00:00,  2.53it/s]


In [ ]:
#documents

In [ ]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from langchain_groq import ChatGroq
from llama_index.core.indices.property_graph import DynamicLLMPathExtractor
from llama_index.core import PropertyGraphIndex
llm = OpenAI(model="gpt-3.5-turbo-0125", temperature=0.0)
#llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
embed_model = OpenAIEmbedding(model_name="text-embedding-3-small")

In [ ]:

kg_extractor = DynamicLLMPathExtractor(
    llm=llm,
    max_triplets_per_chunk=20,
    num_workers=6,
    allowed_entity_types=["MATERIAL", "MATERIAL_TYPE","PART_NUMBER","MANUFACTURER","DIMENSION","PRODUCT"],
    allowed_relation_types=["HAS", "HAS", "PART_OF","MENTIONS","RESPONDS","SIZE_OF","SIZE_IN_INCHES","PRODUCES"],
)

index = PropertyGraphIndex.from_documents(
    documents[:10],
    kg_extractors=[kg_extractor],
    llm=llm,
    embed_model=embed_model,
    property_graph_store=graph_store,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/10 [00:00<?, ?it/s]

Extracting and inferring knowledge graph from text: 100%|██████████| 10/10 [00:05<00:00,  1.72it/s]
Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  1.72it/s]


In [ ]:
from typing import Literal
from llama_index.core.indices.property_graph import SchemaLLMPathExtractor

# best practice to use upper-case
entities = Literal["MANUFACTURER", "MATERIAL", "PRODUCT", "PART NUMBER","MATERIAL_TYPE"]
relations = Literal[
    "HAS_DIMENSION",
    "HAS_MATERIAL",
    "HAS_MATERIAL_TYPE",
    "HAS_PRODUCT",
    "HAS_MANUFACTURER",
    "HAS_SIZE",
    "HAS_COMPANY",
    "HAS_PART_NUMBER",
    "PROVIDES",
]

In [ ]:
# define which entities can have which relations
validation_schema = {
    "Manufacturer": ["HAS_PART_NUMBER"],

    "Part_Number": ["HAS_MANUFACTURER","HAS_DIMENSION", "HAS_SIZE","HAS_PRODUCT","HAS_MATERIAL", "HAS_MATERIAL_TYPE"],
}


In [ ]:
from llama_index.core import PropertyGraphIndex

kg_extractor = SchemaLLMPathExtractor(
    llm=llm,
    possible_entities=entities,
    possible_relations=relations,
    kg_validation_schema=validation_schema,
    # if false, allows for values outside of the schema
    # useful for using the schema as a suggestion
    strict=True,
)

NUMBER_OF_ROWS = 40

index = PropertyGraphIndex.from_documents(
    documents[:NUMBER_OF_ROWS],
    kg_extractors=[kg_extractor],
    llm=llm,
    embed_model=embed_model,
    property_graph_store=graph_store,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/39 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████| 1/1 [00:00<00:00,  1.16it/s]
Generating embeddings: 0it [00:00, ?it/s]


In [ ]:
retriever = index.as_retriever(
    include_text=False,  # include source text in returned nodes, default True
)

nodes = retriever.retrieve("What are part numbers?")

for node in nodes:
    print(node.text)

In [ ]:
from pydantic import BaseModel
from typing import Optional, List


class Entities(BaseModel):
    """List of named entities in the text such as names of material, dimension, size, product, part number, material type, manufacturer of construction industry """
    names: Optional[List[str]]


prompt_template_entities = """
Extract all entities such as material, dimension, size, product, part number, material type, manufacturer of construction industry
from the following text:
{text}
"""

In [ ]:
from typing import Any, Optional

from llama_index.core.embeddings import BaseEmbedding
from llama_index.core.retrievers import CustomPGRetriever, VectorContextRetriever
from llama_index.core.vector_stores.types import VectorStore
from llama_index.program.openai import OpenAIPydanticProgram
from llama_index.core.retrievers import (
    CustomPGRetriever,
    VectorContextRetriever,
    TextToCypherRetriever,
)

class MyCustomRetriever(CustomPGRetriever):
    """Custom retriever with cohere reranking."""

    def init(
        self,
        ## vector context retriever params
        embed_model: Optional[BaseEmbedding] = None,
        vector_store: Optional[VectorStore] = None,
        similarity_top_k: int = 4,
        path_depth: int = 1,
        include_text: bool = True,
        **kwargs: Any,
    ) -> None:
        """Uses any kwargs passed in from class constructor."""
        self.entity_extraction = OpenAIPydanticProgram.from_defaults(
            output_cls=Entities, prompt_template_str=prompt_template_entities
        )
        self.vector_retriever = VectorContextRetriever(
            self.graph_store,
            include_text=self.include_text,
            embed_model=embed_model,
            vector_store=vector_store,  #added
            similarity_top_k=similarity_top_k,
            path_depth=path_depth,
        )
        self.cypher_retriever = TextToCypherRetriever(
            self.graph_store,
            llm=llm,
            #text_to_cypher_template=text_to_cypher_template
            ## NOTE: you can attach other parameters here if you'd like
        )

    def custom_retrieve(self, query_str: str) -> str:
        """Define custom retriever with reranking.

        Could return `str`, `TextNode`, `NodeWithScore`, or a list of those.
        """
        entities = self.entity_extraction(text=query_str).names
        result_nodes = []
        if entities:
            print(f"Detected entities: {entities}")
            for entity in entities:
                result_nodes.extend(self.vector_retriever.retrieve(entity))
        else:
            result_nodes.extend(self.vector_retriever.retrieve(query_str))
        ## TMP: please change
        final_text = "\n\n".join(
            [n.get_content(metadata_mode="llm") for n in result_nodes]
        )
        return final_text

In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine

custom_sub_retriever = MyCustomRetriever(
    index.property_graph_store,
    include_text=True,
    vector_store=index.vector_store,
    embed_model=embed_model
)

query_engine = RetrieverQueryEngine.from_args(
    index.as_retriever(sub_retrievers=[custom_sub_retriever]), llm=llm
)

In [ ]:
response = query_engine.query('provide MUAM400z part number manufacturer')
print(str(response))

Detected entities: ['MUAM400z']
MILWAUKEE VALVE


In [ ]:
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore
from llama_index.core import PropertyGraphIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI



index = PropertyGraphIndex.from_existing(
    property_graph_store=graph_store,
    llm=OpenAI(model="gpt-3.5-turbo", temperature=0.3),
    embed_model=OpenAIEmbedding(model_name="text-embedding-3-small"),
)
# save
index.storage_context.persist("./storage")

In [ ]:
nodes = index.as_retriever(include_text=False).retrieve('provide 3 inch oil seperator part number manufacturer')

print(nodes[0])

In [ ]:
# Print the retrieved nodes' scores
for node in nodes:
    if hasattr(node, 'score'):
        print("Score:", node.score)

Score: 0.7927436828613281
Score: 0.7927436828613281
Score: 0.7819414138793945
Score: 0.7819414138793945
Score: 0.7371950149536133
Score: 0.7002053260803223
Score: 0.7002053260803223


In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine

custom_sub_retriever = MyCustomRetriever(
    index.property_graph_store,
    include_text=True,
    #vector_store=index.vector_store,
    embed_model=embed_model
)

query_engine = RetrieverQueryEngine.from_args(
    index.as_retriever(sub_retrievers=[custom_sub_retriever]), llm=llm
)

In [ ]:
query = 'provide REC970345 part number manufacturer '
#matching all entities, matching atleast 1 or more entities
response = query_engine.query(f'{query}')
print(str(response))
#query_engine.query('clear history')

Detected entities: ['REC970345']
RECTORSEAL CORP THE


In [ ]:
from llama_index.core.graph_stores import (
    SimplePropertyGraphStore,
    EntityNode,
    Relation,
)
from llama_index.core.schema import TextNode

graph_store = SimplePropertyGraphStore()

graph_store
